In [27]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import ShuffleSplit, GridSearchCV,train_test_split
from sklearn.model_selection import cross_validate

from sksurv.datasets import load_veterans_lung_cancer
from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM,FastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

##### In this notebook, we create the different features that we want to put in our evaluation model with a dictionary. After that,
##### we use a gridsearch on SVM survival to find a good set of parameters and we train our model 25 times with the best parameters found
##### using the training and validation set. Finally, we evaluate 25 times our model on a separate testing set.

In [28]:
def score_survival_model(model, X, y):
    prediction = model.predict(X)
    result = concordance_index_censored(y['Status'], y['Survival_in_days'], prediction)
    return result[0]

In [29]:
df_final = pd.read_table("data_frame_final_prognosis.tsv",sep=" ")

eln = [0]
comp =list(range(163,178)) 
age = [162]

all_gen = list(range(1,85))
tmp = df_final.iloc[:,all_gen][df_final.iloc[:,all_gen] >0].count()
gen = [df_final.columns.get_loc(c) for c in tmp[tmp>df_final.shape[0]*0.02].keys() if c in df_final]

all_cyto = list(range(85,154))
tmp = df_final.iloc[:,all_cyto][df_final.iloc[:,all_cyto] >0].count()
cyto = [df_final.columns.get_loc(c) for c in tmp[tmp>df_final.shape[0]*0.02].keys() if c in df_final]

clin =list(range(154,161))
demo = [161,162]
demo_without_age = [161]

eln_comp = eln + comp
eln_age = eln + age
eln_gen = eln + gen
eln_cyto = eln + cyto
eln_clin = eln + clin
eln_demo = eln + demo
eln_demo_without_age = eln + demo_without_age

# USEFUL FOR ELN COMPARISON
# with comp
eln_comp_age = eln_comp + age
eln_comp_gen = eln_comp + gen
eln_comp_cyto = eln_comp + cyto
eln_comp_clin = eln_comp + clin
eln_comp_demo = eln_comp + demo
eln_comp_demo_without_age = eln_comp + demo_without_age

eln_comp_age_gen = eln_comp_age + gen
eln_comp_age_cyto = eln_comp_age + cyto
eln_comp_age_clin = eln_comp_age + clin

eln_comp_gen_cyto = eln_comp_gen + cyto
eln_comp_gen_clin = eln_comp_gen + clin
eln_comp_gen_demo = eln_comp_gen + demo
eln_comp_gen_demo_without_age = eln_comp_gen + demo_without_age

eln_comp_cyto_clin = eln_comp_cyto + clin
eln_comp_cyto_demo = eln_comp_cyto + demo
eln_comp_cyto_demo_without_age = eln_comp_cyto + demo_without_age

eln_comp_clin_demo = eln_comp_clin + demo
eln_comp_clin_demo_without_age = eln_comp_clin + demo_without_age

eln_comp_age_gen_cyto = eln_comp_age_gen + cyto
eln_comp_age_gen_clin = eln_comp_age_gen + clin
eln_comp_age_gen_demo = eln_comp_age_gen + demo


eln_comp_gen_cyto_clin_demo = eln_comp_gen_cyto + clin + demo

# without comp

eln_age_gen = eln_age + gen
eln_age_cyto = eln_age + cyto
eln_age_clin = eln_age + clin

eln_gen_cyto = eln_gen + cyto
eln_gen_clin = eln_gen + clin
eln_gen_demo = eln_gen + demo
eln_gen_demo_without_age = eln_gen + demo_without_age

eln_cyto_clin = eln_cyto + clin
eln_cyto_demo = eln_cyto + demo
eln_cyto_demo_without_age = eln_cyto + demo_without_age

eln_clin_demo = eln_clin + demo
eln_clin_demo_without_age = eln_clin + demo_without_age

eln_age_gen_cyto = eln_age_gen + cyto
eln_age_gen_clin = eln_age_gen + clin
eln_age_gen_demo = eln_age_gen + demo


eln_gen_cyto_clin_demo = eln_gen_cyto + clin + demo

# USEFUL FOR COMP
comp_age = comp + age
comp_gen = comp + gen
comp_cyto = comp + cyto
comp_clin = comp + clin
comp_demo = comp + demo
comp_demo_without_age = comp + demo_without_age
comp_gen_cyto = comp_gen + cyto
comp_clin_demo = comp_clin + demo
comp_gen_cyto_clin_demo = comp_gen_cyto + clin + demo

#USEFUL FOR GEN
gen_age = gen + age
gen_cyto = gen + cyto
gen_clin = gen + clin
gen_demo = gen + demo
gen_demo_without_age = gen + demo_without_age
gen_clin_demo = gen_clin + demo
gen_cyto_clin_demo = gen_cyto + clin + demo

#USEFUL FOR CYTO 
cyto_age = cyto + age
cyto_clin = cyto + clin
cyto_demo = cyto + demo
gen_demo_without_age = gen + demo_without_age
cyto_clin_demo = cyto_clin + demo
cyto_gen_demo = gen_cyto + demo


clin_age  = clin + age


In [30]:
dict_features_type_final_comp = dict(zip(("eln","comp","gen","cyto","clin","demo","eln_comp","eln_age","eln_gen","eln_cyto","eln_clin","eln_demo","eln_demo_without_age","eln_comp_age",
"eln_comp_gen","eln_comp_cyto","eln_comp_clin","eln_comp_demo","eln_comp_demo_without_age","eln_comp_age_gen",
"eln_comp_age_cyto","eln_comp_age_clin","eln_comp_gen_cyto","eln_comp_gen_clin","eln_comp_gen_demo",
"eln_comp_gen_demo_without_age","eln_comp_cyto_clin","eln_comp_cyto_demo","eln_comp_cyto_demo_without_age",
"eln_comp_clin_demo","eln_comp_clin_demo_without_age","eln_comp_age_gen_cyto","eln_comp_age_gen_clin",
"eln_comp_age_gen_demo","eln_comp_gen_cyto_clin_demo","eln_age_gen","eln_age_cyto","eln_age_clin","eln_gen_cyto","eln_gen_clin","eln_gen_demo","eln_gen_demo_without_age",
"eln_cyto_clin","eln_cyto_demo","eln_cyto_demo_without_age","eln_clin_demo","eln_clin_demo_without_age",
"eln_age_gen_cyto","eln_age_gen_clin","eln_age_gen_demo","eln_gen_cyto_clin_demo","comp_age","comp_gen","comp_cyto","comp_clin","comp_demo","comp_demo_without_age","comp_gen_cyto","comp_clin_demo",
 "comp_gen_cyto_clin_demo","gen_age","gen_cyto","gen_clin","gen_demo","gen_demo_without_age","gen_clin_demo","gen_cyto_clin_demo",
 "cyto_age","cyto_clin","cyto_demo","gen_demo_without_age","cyto_clin_demo","cyto_gen_demo","clin_age"),(eln,comp,gen,cyto,clin,demo,eln_comp,eln_age,eln_gen,eln_cyto,eln_clin,eln_demo,eln_demo_without_age,eln_comp_age,
eln_comp_gen,eln_comp_cyto,eln_comp_clin,eln_comp_demo,eln_comp_demo_without_age,eln_comp_age_gen,
eln_comp_age_cyto,eln_comp_age_clin,eln_comp_gen_cyto,eln_comp_gen_clin,eln_comp_gen_demo,
eln_comp_gen_demo_without_age,eln_comp_cyto_clin,eln_comp_cyto_demo,eln_comp_cyto_demo_without_age,
eln_comp_clin_demo,eln_comp_clin_demo_without_age,eln_comp_age_gen_cyto,eln_comp_age_gen_clin,
eln_comp_age_gen_demo,eln_comp_gen_cyto_clin_demo,eln_age_gen,eln_age_cyto,eln_age_clin,eln_gen_cyto,eln_gen_clin,eln_gen_demo,eln_gen_demo_without_age,
eln_cyto_clin,eln_cyto_demo,eln_cyto_demo_without_age,eln_clin_demo,eln_clin_demo_without_age,
eln_age_gen_cyto,eln_age_gen_clin,eln_age_gen_demo,eln_gen_cyto_clin_demo,comp_age,comp_gen,comp_cyto,comp_clin,comp_demo,comp_demo_without_age,comp_gen_cyto,comp_clin_demo,
 comp_gen_cyto_clin_demo,gen_age,gen_cyto,gen_clin,gen_demo,gen_demo_without_age,gen_clin_demo,gen_cyto_clin_demo,
 cyto_age,cyto_clin,cyto_demo,gen_demo_without_age,cyto_clin_demo,cyto_gen_demo,clin_age)))


In [31]:
dict_features_type_final_comp = dict(zip(("eln","comp","gen","cyto","clin","demo","eln_comp","eln_age","eln_gen","eln_cyto","eln_clin","eln_demo","eln_demo_without_age","eln_comp_age",
"eln_comp_gen","eln_comp_cyto","eln_comp_clin","eln_comp_demo","eln_comp_demo_without_age","eln_comp_age_gen",
"eln_comp_age_cyto","eln_comp_age_clin","eln_comp_gen_cyto","eln_comp_gen_clin","eln_comp_gen_demo",
"eln_comp_gen_demo_without_age","eln_comp_cyto_clin","eln_comp_cyto_demo","eln_comp_cyto_demo_without_age",
"eln_comp_clin_demo","eln_comp_clin_demo_without_age","eln_comp_age_gen_cyto","eln_comp_age_gen_clin",
"eln_comp_age_gen_demo","eln_comp_gen_cyto_clin_demo","eln_age_gen","eln_age_cyto","eln_age_clin","eln_gen_cyto","eln_gen_clin","eln_gen_demo","eln_gen_demo_without_age",
"eln_cyto_clin","eln_cyto_demo","eln_cyto_demo_without_age","eln_clin_demo","eln_clin_demo_without_age"),(eln,comp,gen,cyto,clin,demo,eln_comp,eln_age,eln_gen,eln_cyto,eln_clin,eln_demo,eln_demo_without_age,eln_comp_age,
eln_comp_gen,eln_comp_cyto,eln_comp_clin,eln_comp_demo,eln_comp_demo_without_age,eln_comp_age_gen,
eln_comp_age_cyto,eln_comp_age_clin,eln_comp_gen_cyto,eln_comp_gen_clin,eln_comp_gen_demo,
eln_comp_gen_demo_without_age,eln_comp_cyto_clin,eln_comp_cyto_demo,eln_comp_cyto_demo_without_age,
eln_comp_clin_demo,eln_comp_clin_demo_without_age,eln_comp_age_gen_cyto,eln_comp_age_gen_clin,
eln_comp_age_gen_demo,eln_comp_gen_cyto_clin_demo,eln_age_gen,eln_age_cyto,eln_age_clin,eln_gen_cyto,eln_gen_clin,eln_gen_demo,eln_gen_demo_without_age,
eln_cyto_clin,eln_cyto_demo,eln_cyto_demo_without_age,eln_clin_demo,eln_clin_demo_without_age)))
estimator = FastSurvivalSVM(max_iter=1000, tol=1e-6, random_state=17)
param_grid = {'alpha': 10. ** np.array([-6,-5,-4,-3,-2,-1,0]),'optimizer':["avltree"]}
cv = ShuffleSplit(n_splits=5,random_state=17)
gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                   n_jobs=50, iid=False, refit=True,
                   cv=cv)
df=pd.DataFrame(columns=dict_features_type_final_comp.keys())
for key,item in dict_features_type_final_comp.items():
    x = df_final.iloc[:,item]
    y = np.array(list(zip(df_final.os_status, df_final.os)),dtype=[('Status', '?'), ('Survival_in_days', '<f8')])
    ci=[]
    for i in range(25):
        X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(x), y, test_size=0.2, random_state=i)
        gcv = gcv.fit(X_train,y_train)
        print(gcv.best_params_)
        ci.append(concordance_index_censored(y_test['Status'], y_test['Survival_in_days'], gcv.predict(X_test))[0])
        print(ci)
    df[key] = ci
    
df.to_csv("SVM.csv")

{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6089067811212582]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6089067811212582, 0.6039091809721794]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6089067811212582, 0.6039091809721794, 0.6002453585047196]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6089067811212582, 0.6039091809721794, 0.6002453585047196, 0.6328419096221393]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6089067811212582, 0.6039091809721794, 0.6002453585047196, 0.6328419096221393, 0.611692732586146]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6089067811212582, 0.6039091809721794, 0.6002453585047196, 0.6328419096221393, 0.611692732586146, 0.6047662308166509]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6089067811212582, 0.6039091809721794, 0.6002453585047196, 0.6328419096221393, 0.611692732586146, 0.6047662308166509, 0.6258940653392615]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6089067811212582, 0.6039091809721794, 0.6002453585047196, 0.6328419096221393, 0.611692732586146, 0.604766

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.6299832390858253]
{'alpha': 0.001, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668]
{'alpha': 0.01, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668, 0.653403763205601]
{'alpha': 0.01, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668, 0.653403763205601, 0.6509946604923688]
{'alpha': 0.01, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668, 0.653403763205601, 0.6509946604923688, 0.6510324673239346]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668, 0.653403763205601, 0.6509946604923688, 0.6510324673239346, 0.6298546445605269]
{'alpha': 0.01, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668, 0.653403763205601, 0.6509946604923688, 0.6510324673239346, 0.6298546445605269, 0.6354871447902571]
{'alpha': 0.01, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668, 0.653403763205601, 0.6509946604923688, 0.6510324673239346, 0.629854644560

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668, 0.653403763205601, 0.6509946604923688, 0.6510324673239346, 0.6298546445605269, 0.6354871447902571, 0.6188511806357029, 0.6302284212017695, 0.6316701209515266, 0.6448951742959931, 0.6417027764403788]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668, 0.653403763205601, 0.6509946604923688, 0.6510324673239346, 0.6298546445605269, 0.6354871447902571, 0.6188511806357029, 0.6302284212017695, 0.6316701209515266, 0.6448951742959931, 0.6417027764403788, 0.6445276405926398]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668, 0.653403763205601, 0.6509946604923688, 0.6510324673239346, 0.6298546445605269, 0.6354871447902571, 0.6188511806357029, 0.6302284212017695, 0.6316701209515266, 0.6448951742959931, 0.6417027764403788, 0.6445276405926398, 0.6415833319956982]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.6299832390858253, 0.6243645753820668, 0.653403763

...

{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.7048982596845189, 0.6942328618063112, 0.6802056635619178, 0.7296382050433934, 0.7045987347056746, 0.6932773109243697, 0.7056833558863329, 0.6962163511026545, 0.6865229916723077, 0.6898722430166734, 0.7126597196377621, 0.6918472155352271, 0.6883224470288354, 0.7064479365639897, 0.7085628206938757, 0.707140365896192, 0.7023830538393645]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7048982596845189, 0.6942328618063112, 0.6802056635619178, 0.7296382050433934, 0.7045987347056746, 0.6932773109243697, 0.7056833558863329, 0.6962163511026545, 0.6865229916723077, 0.6898722430166734, 0.7126597196377621, 0.6918472155352271, 0.6883224470288354, 0.7064479365639897, 0.7085628206938757, 0.707140365896192, 0.7023830538393645, 0.6818649848767617]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.7048982596845189, 0.6942328618063112, 0.6802056635619178, 0.7296382050433934, 0.7045987347056746, 0.6932773109243697, 0.7056833558863329, 0.6962163511026545, 0.6865229916723

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.6300302323031376, 0.6247705975021519, 0.622179158592236, 0.6438831923640316, 0.6283197919008318, 0.6271616754810032, 0.6352696694374638, 0.6024595277457189, 0.6342505864017758, 0.6168914529650138, 0.6320788363726585, 0.6015487080725406, 0.6379958578938983, 0.6378673814989004, 0.6258453765161858, 0.634920257459019, 0.6491936130947605, 0.6381845678615098, 0.6059131501163448, 0.6310069934556654, 0.6278433561971606, 0.5976178683836334, 0.634762011101895, 0.6268609984814566, 0.6462535192716137]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7109917135293473]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.7109917135293473, 0.7145177269257629]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.7109917135293473, 0.7145177269257629, 0.679580546352441]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.7109917135293473, 0.7145177269257629, 0.679580546352441, 0.734552442155334]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.7109917135293473, 0.7145177269257629, 0.6795805463

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 0.001, 'optimizer': 'avltree'}
[0.6524068359466784, 0.6730222662530655, 0.655544789648059, 0.6809368552032635, 0.6400815697357012, 0.6647415723045975, 0.6656195631161802, 0.6491780779188372, 0.6443650331376037, 0.6230550313979026, 0.640863416449572, 0.6344567485154871, 0.6664170782220806, 0.6689192442896355]
{'alpha': 0.01, 'optimizer': 'avltree'}
[0.6524068359466784, 0.6730222662530655, 0.655544789648059, 0.6809368552032635, 0.6400815697357012, 0.6647415723045975, 0.6656195631161802, 0.6491780779188372, 0.6443650331376037, 0.6230550313979026, 0.640863416449572, 0.6344567485154871, 0.6664170782220806, 0.6689192442896355, 0.6482292206060035]
{'alpha': 0.001, 'optimizer': 'avltree'}
[0.6524068359466784, 0.6730222662530655, 0.655544789648059, 0.6809368552032635, 0.6400815697357012, 0.6647415723045975, 0.6656195631161802, 0.6491780779188372, 0.6443650331376037, 0.6230550313979026, 0.640863416449572, 0.6344567485154871, 0.6664170782220806, 0.6689192442896355, 0.6482292206060035, 0

In [32]:
dict_features_type_final_comp = dict(zip(("eln_age_gen_cyto","eln_age_gen_clin","eln_age_gen_demo","eln_gen_cyto_clin_demo","comp_age","comp_gen","comp_cyto","comp_clin","comp_demo","comp_demo_without_age","comp_gen_cyto","comp_clin_demo",
 "comp_gen_cyto_clin_demo","gen_age","gen_cyto","gen_clin","gen_demo","gen_demo_without_age","gen_clin_demo","gen_cyto_clin_demo",
 "cyto_age","cyto_clin","cyto_demo","gen_demo_without_age","cyto_clin_demo","cyto_gen_demo","clin_age"),(eln_age_gen_cyto,eln_age_gen_clin,eln_age_gen_demo,eln_gen_cyto_clin_demo,comp_age,comp_gen,comp_cyto,comp_clin,comp_demo,comp_demo_without_age,comp_gen_cyto,comp_clin_demo,
 comp_gen_cyto_clin_demo,gen_age,gen_cyto,gen_clin,gen_demo,gen_demo_without_age,gen_clin_demo,gen_cyto_clin_demo,
 cyto_age,cyto_clin,cyto_demo,gen_demo_without_age,cyto_clin_demo,cyto_gen_demo,clin_age)))

estimator = FastSurvivalSVM(max_iter=1000, tol=1e-6, random_state=17)
param_grid = {'alpha': 10. ** np.array([-6,-5,-4,-3,-2,-1,0]),'optimizer':["avltree"]}
cv = ShuffleSplit(n_splits=5,random_state=17)
gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                   n_jobs=50, iid=False, refit=True,
                   cv=cv)

df1=pd.DataFrame(columns=dict_features_type_final_comp.keys())

for key,item in dict_features_type_final_comp.items():
    x = df_final.iloc[:,item]
    y = np.array(list(zip(df_final.os_status, df_final.os)),dtype=[('Status', '?'), ('Survival_in_days', '<f8')])
    ci=[]
    for i in range(25):
        X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(x), y, test_size=0.2, random_state=i)
        gcv = gcv.fit(X_train,y_train)
        print(gcv.best_params_)
        ci.append(concordance_index_censored(y_test['Status'], y_test['Survival_in_days'], gcv.predict(X_test))[0])
        print(ci)
    df1[key] = ci
    
df1.to_csv("SVM_add.csv")




{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7134353608295869]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.7134353608295869, 0.6926250142107742]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7134353608295869, 0.6926250142107742, 0.6836906920047509]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7134353608295869, 0.6926250142107742, 0.6836906920047509, 0.732315834239002]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7134353608295869, 0.6926250142107742, 0.6836906920047509, 0.732315834239002, 0.7044542213943928]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7134353608295869, 0.6926250142107742, 0.6836906920047509, 0.732315834239002, 0.7044542213943928, 0.6932935336296681]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7134353608295869, 0.6926250142107742, 0.6836906920047509, 0.732315834239002, 0.7044542213943928, 0.6932935336296681, 0.7055705908885882]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.7134353608295869, 0.6926250142107742, 0.6836906920047509, 0.732315834239002, 0.7044542213943928, 0.693293

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 0.001, 'optimizer': 'avltree'}
[0.6632152759285076, 0.6774235460347879, 0.6901606551228355, 0.6918678826253367, 0.6641831786505669, 0.6792771162519061, 0.6711933758618468, 0.6681743036276865, 0.6548651669474049, 0.652797970736536, 0.6621386924699169, 0.6697801316000642, 0.6815198343157559, 0.6751793768760334, 0.6681259709080638, 0.6839310060559942, 0.682099013078713, 0.6725497136237854, 0.6495346204641218, 0.6502149364814577, 0.6658159196044026, 0.6479366043041331, 0.7014770624641102, 0.6465394429920003, 0.6633682928033983]


/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7125268252948824]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.7125268252948824, 0.6989914410537086]
{'alpha': 1.0, 'optimizer': 'avltree'}
[0.7125268252948824, 0.6989914410537086, 0.6869413014940301]
{'alpha': 0.001, 'optimizer': 'avltree'}
[0.7125268252948824, 0.6989914410537086, 0.6869413014940301, 0.7283781442454599]
{'alpha': 0.001, 'optimizer': 'avltree'}
[0.7125268252948824, 0.6989914410537086, 0.6869413014940301, 0.7283781442454599, 0.7052891871929092]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.7125268252948824, 0.6989914410537086, 0.6869413014940301, 0.7283781442454599, 0.7052891871929092, 0.6897569838746309]
{'alpha': 0.01, 'optimizer': 'avltree'}
[0.7125268252948824, 0.6989914410537086, 0.6869413014940301, 0.7283781442454599, 0.7052891871929092, 0.6897569838746309, 0.7034280559314389]
{'alpha': 0.1, 'optimizer': 'avltree'}
[0.7125268252948824, 0.6989914410537086, 0.6869413014940301, 0.7283781442454599, 0.7052891871929092, 0.68975

...

{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7138896285969392, 0.6898478229093921, 0.6820966431205852, 0.7280473782860023, 0.7067824914094865, 0.6931961973978781, 0.7038791159224177, 0.6977603793006644, 0.6846968814445161, 0.6941720543199184, 0.7170791465078774, 0.6965655593002729, 0.6846264138919866, 0.7014558821168879, 0.7093160099797581, 0.7143378039887124, 0.7067319265024472, 0.6833129545015767, 0.6777246714043142, 0.6963300397792891, 0.6893762960599776, 0.6780687319263153]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7138896285969392, 0.6898478229093921, 0.6820966431205852, 0.7280473782860023, 0.7067824914094865, 0.6931961973978781, 0.7038791159224177, 0.6977603793006644, 0.6846968814445161, 0.6941720543199184, 0.7170791465078774, 0.6965655593002729, 0.6846264138919866, 0.7014558821168879, 0.7093160099797581, 0.7143378039887124, 0.7067319265024472, 0.6833129545015767, 0.6777246714043142, 0.6963300397792891, 0.6893762960599776, 0.6780687319263153, 0.7119887704970331]
{'alpha': 1e-05,

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7138896285969392, 0.6898478229093921, 0.6820966431205852, 0.7280473782860023, 0.7067824914094865, 0.6931961973978781, 0.7038791159224177, 0.6977603793006644, 0.6846968814445161, 0.6941720543199184, 0.7170791465078774, 0.6965655593002729, 0.6846264138919866, 0.7014558821168879, 0.7093160099797581, 0.7143378039887124, 0.7067319265024472, 0.6833129545015767, 0.6777246714043142, 0.6963300397792891, 0.6893762960599776, 0.6780687319263153, 0.7119887704970331, 0.7020523819215054, 0.6944531340039843]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.6919594605178653]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.6919594605178653, 0.6993487405193834]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6919594605178653, 0.6993487405193834, 0.6588891667187597]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.6919594605178653, 0.6993487405193834, 0.6588891667187597, 0.7059333112822693]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.6919594605178653, 0.6993487405193834, 0.658889

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6919594605178653, 0.6993487405193834, 0.6588891667187597, 0.7059333112822693, 0.6760011561064903, 0.6844034911261802, 0.6889296990785488, 0.6985089990330329, 0.6590683689372353, 0.668759860178798, 0.6803126163007072, 0.6822981864869202, 0.6690138601242632, 0.6877799001589111, 0.68803841265358, 0.7141000031706776, 0.686720693252026, 0.6456174786022266, 0.6721747059933337, 0.6776915180290004, 0.6811931727548254]
{'alpha': 0.1, 'optimizer': 'avltree'}
[0.6919594605178653, 0.6993487405193834, 0.6588891667187597, 0.7059333112822693, 0.6760011561064903, 0.6844034911261802, 0.6889296990785488, 0.6985089990330329, 0.6590683689372353, 0.668759860178798, 0.6803126163007072, 0.6822981864869202, 0.6690138601242632, 0.6877799001589111, 0.68803841265358, 0.7141000031706776, 0.686720693252026, 0.6456174786022266, 0.6721747059933337, 0.6776915180290004, 0.6811931727548254, 0.6538000671022991]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.6919594605178653, 0.6

In [33]:
df.join(df1).to_csv("SVM.csv")